**Game Theory Model for Multichannel Marketing Attribution**

* https://medium.com/data-from-the-trenches/marketing-attribution-e7fa7ae9e919
* https://recercat.cat/bitstream/handle/2072/290758/201702.pdf

The use of cooperative game theory to attribute a fair value to all channels that contribute to a sales conversion. In this context, a game is defined by a set of channels $N$ and a characteristic function $v$. Each subset of channels in $N$ is called a coalition $A$, and the characteristic function assigns a value to every coalition to signify it's overall worth. A coalition's worth represents the payoff that it can generate when its channels work together.

Options for defining the characteristic function include:
* $v(A)$ is the conditional probability of conversion for each coalition. Estimated by fitting a predictive model on past coalition performance.
* $v(A)$ is simply the total number of conversions generated by each coalition. 

This workbook implements the simpler conversion count characteristic function.

In [1]:
from itertools import combinations

def subsets(S):
    '''Returns all possible subsets of the given set'''
    s = []
    for i in range(1, len(S)+1):
        s.extend(map(list, combinations(S, i)))
    return list(map('+'.join, s))

In [2]:
# N is the set of channels active during the reporting period (the set of players in the game).
N = sorted({'1', '2', '3'})

coalitions = subsets(N)
print('Channels: {}'.format(coalitions))

Channels: ['1', '2', '3', '1+2', '1+3', '2+3', '1+2+3']


In [4]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.0f}'.format

# I(R) is the array of coalition frequencies.
# For example, the coalition containing channels 1+2 has generated 898 conversions
IR = np.array([19786, 20837, 24008, 898, 822, 822, 194])

IRx = ['I({})'.format(S.replace('+', '')) for S in coalitions]
pd.DataFrame({
    'Coalition': coalitions,
    'Frequency': IR
}, dtype=np.float64, index=IRx)

,Coalition,Frequency
I(1),1,"19,786"
I(2),2,"20,837"
I(3),3,"24,008"
I(12),1+2,898
I(13),1+3,822
I(23),2+3,822
I(123),1+2+3,194


In [5]:
print('Total conversions: {:,}'.format(IR.sum()))

Total conversions: 67,367


In [6]:
import numpy as np

# B is binary squared matrix that represents coalition membership.
# For example, coalition '1+2' includes members 1, 2, and 1+2, giving
# the coefficients [1,1,0,1,0,0,0]

d = 2**len(N)-1
B = np.matrix(np.zeros((d, d)))

for i in range(0, d):
    A = coalitions[i]
    S = subsets(A.split('+'))
    coef = [1 if c in S else 0 for c in coalitions]
    B[i] = coef

pd.DataFrame(data=B, index=coalitions, columns=coalitions)

,1,2,3,1+2,1+3,2+3,1+2+3
1,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0
1+2,1,1,0,1,0,0,0
1+3,1,0,1,0,1,0,0
2+3,0,1,1,0,0,1,0
1+2+3,1,1,1,1,1,1,1


In [7]:
# The product of the matrices coalition membership 'B' and coalition frequencies 'I(R)'
# is the coalition worth 'v(S)' - the result of the characteristic function.

# For example:
# - v(12) = I(1) + I(2) + I(12)
# - v(12) = 19,786 + 20,837 + 898
# - v(12) = 41,521

vS = np.dot(B, IR)
vS = np.squeeze(np.asarray(vS))

vSx = ['v({})'.format(S.replace('+', '')) for S in coalitions]
pd.DataFrame({
    'Coalition': coalitions,
    'Worth': vS
}, index=vSx)

,Coalition,Worth
v(1),1,"19,786"
v(2),2,"20,837"
v(3),3,"24,008"
v(12),1+2,"41,521"
v(13),1+3,"44,616"
v(23),2+3,"45,667"
v(123),1+2+3,"67,367"


In [8]:
from collections import defaultdict
from math import factorial

# Calculate the Shapley values - the average value of each channel's marginal contribution
# to the grand coalition, taking into account all possible orderings.

shapley = defaultdict(int)
n = len(N)

for i in N:
    for A in coalitions:
        S = A.split('+')
        if i not in S:
            k = len(S) # Cardinality of set |S|
            Si = S
            Si.append(i)
            Si = '+'.join(sorted(Si))
            # Weight = |S|!(n-|S|-1)!/n!
            weight = (factorial(k) * factorial(n-k-1)) / factorial(n)
            # Marginal contribution = v(S U {i})-v(S)
            contrib = vS[coalitions.index(Si)] - vS[coalitions.index(A)]            
            shapley[i] += weight * contrib
    shapley[i] += vS[coalitions.index(i)]/n

In [9]:
n_conversions = sum(shapley.values())
p_shapley = [100*sv/n_conversions for sv in shapley.values()]

pd.DataFrame({
    'Channel': list(shapley.keys()),
    'Shapley value': list(shapley.values()),
    'Shapley percent': p_shapley
})

,Channel,Shapley value,Shapley percent
0,1,"20,711",31
1,2,"21,762",32
2,3,"24,895",37
